In [1]:
from datetime import timedelta

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA

from sklearn.metrics import r2_score

import warnings

### Task_1:

1.	Прочитайте базу my_PJME_MW.csv и сделайте ее перерасчет (ресемплинг) в представление по неделям.

In [2]:
df = pd.read_csv('my_PJME_MW.csv', parse_dates=['Date'], sep=',')
df.index = df['Date']

df = df.resample('W').mean()
df.head()

,PJME_MW,year,month,dayofmonth,quarter,dayofyear,weekofyear,dayofweek,hour,is_holiday,is_prev_holiday,is_next_holiday
Date,,,,,,,,,,,,
2002-01-06,32392.160839,2002.0,1.000000,3.517483,1.0,3.517483,1.0,3.517483,11.58042,0.160839,0.000000,0.167832
2002-01-13,31487.386905,2002.0,1.000000,10.000000,1.0,10.000000,2.0,3.000000,11.50000,0.000000,0.000000,0.000000
2002-01-20,31452.880952,2002.0,1.000000,17.000000,1.0,17.000000,3.0,3.000000,11.50000,0.000000,0.142857,0.000000
2002-01-27,30141.529762,2002.0,1.000000,24.000000,1.0,24.000000,4.0,3.000000,11.50000,0.142857,0.000000,0.142857
2002-02-03,29196.488095,2002.0,1.428571,17.714286,1.0,31.000000,5.0,3.000000,11.50000,0.000000,0.000000,0.000000


### Task_2:

2.	Постройте модель предсказания 4-й точки от текущей (h = 4), используя результаты автокорреляционного анализа из предшествующих уроков .

In [3]:
# warnings.filterwarnings("ignore")

model = sm.tsa.statespace.SARIMAX(df['PJME_MW'].values[:-5], 
                                order = [0,0,1],
                                seasonal_order = [0,1,2,52],
                                enforce_stationarity = False,
                                enforce_invertibility = False, n_jobs=10)
model_fitted = model.fit()
print(f'AIC: {model_fitted.aic}')

AIC: 12968.415447164662


In [4]:
pred_4 = model_fitted.forecast(4)

### Task_3:

3.	Постройте модель предсказания 1-й точки от текущей (h = 1), используя результаты автокорреляционного анализа из предшествующих уроков.

In [5]:
true_total = df['PJME_MW']
true_4 = df.loc[df.index[-1], "PJME_MW"]

In [6]:
warnings.filterwarnings("ignore")

model = sm.tsa.statespace.SARIMAX(df['PJME_MW'].values[:-1], 
                                order = [0,0,1],
                                seasonal_order = [0,1,2,52],
                                enforce_stationarity = False,
                                enforce_invertibility = False, n_jobs=10)
model_fitted = model.fit()
print(f'AIC: {model_fitted.aic}')

AIC: 13005.810123014558


In [7]:
pred_1 = model_fitted.forecast(1)

### Task_4:

4.	Примените авторекурсию и сравните результат в 4-й точке путем прямого моделирования и путем рекурсивного моделирования на длине выборки из 4-х точек.

In [8]:
for i in range(1, 5):
    model = sm.tsa.statespace.SARIMAX(df['PJME_MW'].values[:-5], 
                                order = [0,0,1],
                                seasonal_order = [0,1,2,52],
                                enforce_stationarity = False,
                                enforce_invertibility = False)
    model_fitted = model.fit()
    pred = model_fitted.forecast(1)
    print(f'Pred for pred {i} done, value: {pred}.')
    
    df.loc[df.index[-1] + timedelta(days=7 * i), 'PJME_MW'] = pred

Pred for pred 1 done, value: [36895.65219772].
Pred for pred 2 done, value: [38104.68699654].
Pred for pred 3 done, value: [38969.01493888].
Pred for pred 4 done, value: [35918.01727898].


In [9]:
print(f'True: {true_4}')
print(f'Predict на 4 вперед точечный: {pred_4[-1]}')
print(f'Predict на 4 вперед рекурсивный: {df.loc[df.index[-1], "PJME_MW"]}')

True: 36568.80412371134
Predict на 4 вперед точечный: [36895.65219772 37575.70192158 39593.61702802 37070.72442598]
Predict на 4 вперед рекурсивный: 35918.017278977066


In [10]:
'r2 для не рекурсивного предсказания на 4 шага', r2_score(true_total[-4:], pred_4)

('r2 для не рекурсивного предсказания на 4 шага', -19.970353705111485)

In [11]:
'r2 для рекурсивного предсказания на 4 шага', r2_score(true_total[-4:], df.loc[df.index[-4:], "PJME_MW"])

('r2 для рекурсивного предсказания на 4 шага', -18.651011792907184)

### Task_5:

5.	Оцените рост ошибки прогнозирования рекурсивной модели в интервалах от 1-й до 10-й, от 10-й до 20-й, от 20-й до 30-й, .. .10*i-й до (i+1)*10 -й,... ,от 90 до 100-й точках (используйте осреднение по десяткам точек)

In [12]:
for i in range(1, 100):
    model = sm.tsa.statespace.SARIMAX(df['PJME_MW'].values[:-100], 
                                order = [0,0,1],
                                seasonal_order = [0,1,2,52],
                                enforce_stationarity = False,
                                enforce_invertibility = False)
    model_fitted = model.fit()
    pred = model_fitted.forecast(1)
    print(f'Pred for pred {i} done, value: {pred}.')
    
    df.loc[df.index[-1] + timedelta(days=7 * i), 'PJME_MW'] = pred

Pred for pred 1 done, value: [25452.80242616].
Pred for pred 2 done, value: [27124.37689768].
Pred for pred 3 done, value: [26361.10907992].
Pred for pred 4 done, value: [27524.80456021].
Pred for pred 5 done, value: [27337.6926393].
Pred for pred 6 done, value: [28370.02985045].
Pred for pred 7 done, value: [30012.0046613].
Pred for pred 8 done, value: [28889.10810456].
Pred for pred 9 done, value: [30489.88689029].
Pred for pred 10 done, value: [31399.56975464].
Pred for pred 11 done, value: [31745.66787738].
Pred for pred 12 done, value: [29887.1204423].
Pred for pred 13 done, value: [32207.67596376].
Pred for pred 14 done, value: [34955.82631459].
Pred for pred 15 done, value: [33580.50167277].
Pred for pred 16 done, value: [35881.91055884].
Pred for pred 17 done, value: [33208.61489455].
Pred for pred 18 done, value: [32811.34726973].
Pred for pred 19 done, value: [36642.51321297].
Pred for pred 20 done, value: [34229.65221279].
Pred for pred 21 done, value: [33321.85080261].
Pred

KeyboardInterrupt: 

Вычисляю качество предиктов на "десятках" шагов вперед при рекурсивном подходе:

In [ ]:
mean_errors = []

for i in sorted(range(10), reverse=True):
    try:
        r2 = r2_score(y_true=true_total[-(i+1) * 10 : -(i) * 10],
                      y_pred=df.loc[df.index[-(i+1) * 10 : -(i) * 10], "PJME_MW"])
        mean_errors.append(r2)
    except:
        pass

mean_errors
errors = pd.DataFrame({'error': mean_errors})
errors['MA'] = errors['error'].rolling(5).mean()

plt.plot(errors['MA'])    

### Task_6:

6.	Сделайте вывод о том, как изменилось поведение ошибки предсказания ряда.

##### Выводы:

Вычисления ARIMA почему-то долго шли, сложно было дебажить, не успел додебажить - почему-то пока r2 по всем прогнозам отрицательный (выглядит как проявление неотловленного бага), но могу спрогонзировать чем бы закончилось: рекурсивная модель накапливала бы в себе ошибки и в среднем на каждом новом десятке (как отражение общей тенденции) средняя ошибка бы росла. Т.е. ошибка бы закладывалась в данные, на которые опирается каждая последующая модель, ошибка бы нарастала. Приращение ошибки на шаге были бы похожи на белый шум, а накопленная ошибка - в виде случайного блуждания, именно поэтому нарастание ошибки предсказаний.